# 0.2.0 Recency, Frequency, Monetary Value analysis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

import utils.paths as path
from utils.paths2 import direcciones

## 0.2.2 Calculate spend quartiles (q=4)

In [4]:
online = pd.read_csv(path.data_raw_dir("online.csv"), sep=',')
online.drop(['Unnamed: 0'], axis=1, inplace=True)
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'], format='%Y-%m-%d %H:%M:%S') #, dayfirst='true', errors = 'coerce'
print(online.info())
online.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70864 entries, 0 to 70863
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    70864 non-null  int64         
 1   StockCode    70864 non-null  object        
 2   Description  70864 non-null  object        
 3   Quantity     70864 non-null  int64         
 4   InvoiceDate  70864 non-null  datetime64[ns]
 5   UnitPrice    70864 non-null  float64       
 6   CustomerID   70864 non-null  int64         
 7   Country      70864 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 4.3+ MB
None


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom
1,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom
2,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom
3,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom
4,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom


In [5]:
data = pd.read_csv(path.data_interim_dir("data.csv"), sep=',')
data

,CustomerID,Spend,Recency_Days
0,0,137,37
1,1,335,235
2,2,172,396
3,3,355,72
4,4,303,255
5,5,233,393
6,6,244,203
7,7,229,133


In [6]:
# Create a spend quartile with 4 groups - a range between 1 and 5
spend_quartile = pd.qcut(data['Spend'], q=4, labels=range(1,5))

# Assign the quartile values to the Spend_Quartile column in data
data['Spend_Quartile'] = spend_quartile

# Print data with sorted Spend values
print(data.sort_values('Spend'))

   CustomerID  Spend  Recency_Days Spend_Quartile
0           0    137            37              1
2           2    172           396              1
7           7    229           133              2
5           5    233           393              2
6           6    244           203              3
4           4    303           255              3
1           1    335           235              4
3           3    355            72              4


## 0.2.3 Calculate recency deciles (q=4)

In [7]:
# Store labels from 4 to 1 in a decreasing order
r_labels = list(range(4, 0, -1))

# Create a spend quartile with 4 groups and pass the previously created labels 
recency_quartiles = pd.qcut(data['Recency_Days'], q=4, labels=r_labels)

# Assign the quartile values to the Recency_Quartile column in `data`
data['Recency_Quartile'] = recency_quartiles 

# Print `data` with sorted Recency_Days values
print(data.sort_values('Recency_Days'))

   CustomerID  Spend  Recency_Days Spend_Quartile Recency_Quartile
0           0    137            37              1                4
3           3    355            72              4                4
7           7    229           133              2                3
6           6    244           203              3                3
1           1    335           235              4                2
4           4    303           255              3                2
5           5    233           393              2                1
2           2    172           396              1                1


## 0.2.5 Largest frequency value

In [8]:
datamart = pd.read_csv(path.data_raw_dir("datamart.csv"), sep=',', index_col='CustomerID')

In [9]:
datamart['Frequency'].mean()

18.71424650013725

What is the average Frequency value?

R:/ 18.71424650013725

## 0.2.6 Calculate RFM values

TotalSum = online['Quantity'] * online['UnitPrice']

In [10]:
online = pd.read_csv(path.data_raw_dir("online2.csv"), sep=',')
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'])#, format='%Y-%m-%d %H:%M:%S') #, dayfirst='true', errors = 'coerce'
print(online.info())
online.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68170 entries, 0 to 68169
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    68170 non-null  int64         
 1   StockCode    68170 non-null  object        
 2   Description  68170 non-null  object        
 3   Quantity     68170 non-null  int64         
 4   InvoiceDate  68170 non-null  datetime64[ns]
 5   UnitPrice    68170 non-null  float64       
 6   CustomerID   68170 non-null  int64         
 7   TotalSum     68170 non-null  float64       
 8   Country      68170 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 4.7+ MB
None


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,TotalSum,Country
0,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25,2.10,14286,12.60,United Kingdom
1,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20,1.45,16360,1.45,United Kingdom
2,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14,3.75,13933,22.50,United Kingdom
3,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23,2.10,17290,2.10,United Kingdom
4,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25,5.95,17663,17.85,United Kingdom


In [11]:
snapshot_date = pd.Timestamp(2011, 12, 10, 0)
snapshot_date

Timestamp('2011-12-10 00:00:00')

In [12]:
# Calculate Recency, Frequency and Monetary value for each customer 
datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})

In [13]:
# Rename the columns 
datamart.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'TotalSum': 'MonetaryValue'}, inplace=True)

# Print top 5 rows
print(datamart.head())

            Recency  Frequency  MonetaryValue
CustomerID                                   
12747            23         25         948.70
12748             5        888        7046.16
12749            23         37         813.45
12820            45         17         268.02
12822            71          9         146.15


## 0.2.7 Calculate 3 groups for recency and frequency

In [14]:
# Create labels for Recency and Frequency
r_labels = range(3, 0, -1); f_labels = range(1, 4)

# Assign these labels to three equal percentile groups 
r_groups = pd.qcut(datamart['Recency'], q=3, labels=r_labels)

# Assign these labels to three equal percentile groups 
f_groups = pd.qcut(datamart['Frequency'], q=3, labels=f_labels)

# Create new columns R and F 
datamart = datamart.assign(R=r_groups.values, F=f_groups.values)

## 0.2.8 Calculate RFM Score

In [15]:
# Create labels for MonetaryValue
m_labels = range(1, 4)

# Assign these labels to three equal percentile groups 
m_groups = pd.qcut(datamart['MonetaryValue'], q=3, labels=m_labels)

# Create new column M
datamart = datamart.assign(M=m_groups.values)

# Calculate RFM_Score
datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)
print(datamart['RFM_Score'].head())

CustomerID
12747    9
12748    9
12749    9
12820    8
12822    6
Name: RFM_Score, dtype: int64


## 0.2.10 Find average value for RFM score segment

In [16]:
# Create labels for Recency and Frequency
r_labels = range(4, 0, -1); f_labels = range(1, 5)

# Assign these labels to three equal percentile groups 
r_groups = pd.qcut(datamart['Recency'], q=4, labels=r_labels)

# Assign these labels to three equal percentile groups 
f_groups = pd.qcut(datamart['Frequency'], q=4, labels=f_labels)

# Create new columns R and F 
datamart = datamart.assign(R=r_groups.values, F=f_groups.values)

In [17]:
# Create labels for MonetaryValue
m_labels = range(1, 5)

# Assign these labels to three equal percentile groups 
m_groups = pd.qcut(datamart['MonetaryValue'], q=4, labels=m_labels)

# Create new column M
datamart = datamart.assign(M=m_groups.values)

# Calculate RFM_Score
def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])
datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)
datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)

In [18]:
datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score,RFM_Segment
CustomerID,,,,,,,,
12747,23,25,948.70,4,4,4,12,4.04.04.0
12748,5,888,7046.16,4,4,4,12,4.04.04.0
12749,23,37,813.45,4,4,4,12,4.04.04.0
12820,45,17,268.02,3,3,3,9,3.03.03.0
12822,71,9,146.15,2,2,3,7,2.02.03.0


In [19]:
datamart.groupby('RFM_Score').agg({'MonetaryValue': ['mean'] }).round(1).loc[9]

MonetaryValue  mean    321.8
Name: 9, dtype: float64

The average MonetaryValue is 321.8

## 0.2.11 Creating custom segments

In [20]:
# Define rfm_level function
def rfm_level(df):
    if df['RFM_Score'] >= 10:
        return 'Top'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 10)):
        return 'Middle'
    else:
        return 'Low'

# Create a new variable RFM_Level
datamart['RFM_Level'] = datamart.apply(rfm_level, axis=1)

# Print the header with top 5 rows to the console
print(datamart.head())

            Recency  Frequency  MonetaryValue  R  F  M  RFM_Score RFM_Segment  \
CustomerID                                                                      
12747            23         25         948.70  4  4  4         12   4.04.04.0   
12748             5        888        7046.16  4  4  4         12   4.04.04.0   
12749            23         37         813.45  4  4  4         12   4.04.04.0   
12820            45         17         268.02  3  3  3          9   3.03.03.0   
12822            71          9         146.15  2  2  3          7   2.02.03.0   

           RFM_Level  
CustomerID            
12747            Top  
12748            Top  
12749            Top  
12820         Middle  
12822         Middle  


## 0.2.12 Analyzing custom segments

In [21]:
# Calculate average values for each RFM_Level, and return a size of each segment 
rfm_level_agg = datamart.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

# Print the aggregated dataset
print(rfm_level_agg)

          Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Level                                      
Low         191.2       3.3          52.2  1088
Middle       85.8      10.8         206.0  1526
Top          26.4      46.8         951.6  1029


In [22]:
print('ok_')

ok_
